# Visualization

In [ ]:
import pandas as pd
import numpy as np
import os, cv2, copy

from PIL import Image, ImageFont, ImageDraw, ImageEnhance
import matplotlib.cm as mpl_color_map
import matplotlib.pyplot as plt

In [ ]:
from ast import literal_eval

In [ ]:
from matplotlib.colors import LinearSegmentedColormap

# get colormap
ncolors = 256
color_array = plt.get_cmap('gist_gray')(range(ncolors))

# change alpha values
color_array[:,-1] = np.linspace(1.0,0.0,ncolors)

# create a colormap object
map_object = LinearSegmentedColormap.from_list(name='gray_alpha',colors=color_array)

# register this new colormap with matplotlib
plt.register_cmap(cmap=map_object)

In [ ]:
colors = [(44,160,44),(31,119,180),(255,127,14),(214,39,40),(148,103,189),
         (140,86,75),(227,119,194),(127,127,127),(188,189,34),(255,152,150),
         (23,190,207),(174,199,232),(255,187,120),(152,223,138),(197,176,213),
         (196,156,148),(247,182,210),(199,199,199),(219,219,141),(158,218,229),
         (57,59,121),(82,84,163),(107,110,207),(156,158,222),(99,121,57),
         (140,162,82),(181,207,107),(206,219,156),(140,109,49),(189,158,57),
         (231,186,82),(231,203,148),(132,60,57),(173,73,74),(214,97,107),
         (123,65,115),(165,81,148),(206,109,189),(222,158,214),(49,130,189),
         (107,174,214),(158,202,225),(198,219,239),(230,85,13),(253,141,60),]

In [ ]:
emotions = ['amusement', 'awe', 'contentment', 'excitement', 'anger', 'disgust', 'fear', 'sadness']
emotion_dict = {
    "amusement": 0, "awe": 1, "contentment": 2, "excitement": 3,
    "anger": 4, "disgust": 5, "fear": 6, "sadness": 7,
}

In [ ]:
img_root = './data/artworks/'
bbox_root = './data/arts_features_vinvl_bbox_col/'
heatmap_root = './data/arts_features_vinvl_heatmap_sum_unified/'

In [ ]:
test_file = './data/test.json'
df_test = pd.read_json(test_file)

In [ ]:
test_result_file = './data/test_result.json'
df_test_pred = pd.read_json(test_result_file)
df_test_pred

In [ ]:
df_apolo = pd.read_json('./data/apolo.json')
df_apolo

In [ ]:
num_boxes = []
box_cords = []

for i in range(df_test.shape[0]):
    file_name = df_test.iloc[i]['painting']# .replace('eugã¨ne','eugene')
    img_file = os.path.join(img_root,file_name+'.jpg')
    img = Image.open(img_file)
    w, h = img.size
    
    bboxes = np.load(bbox_root+file_name.replace('eugã¨ne','eugene')+'.npy', allow_pickle=True)
    
    box_cord = []
    for x in range(bboxes.shape[0]):
        bbox = bboxes[x]
        x1 = int(bbox[0] * w)
        y1 = int(bbox[1] * h)
        x2 = int(bbox[2] * w)
        y2 = int(bbox[3] * h)
        box_cord.append([x1,y1,x2,y2])
    
    num_boxes.append(bboxes.shape[0])
    box_cords.append(box_cord)

In [ ]:
df_test['num_boxes'] = num_boxes
df_test['box_cords'] = box_cords
df_test

In [ ]:
# change this index to view more annotations
idx = 0

In [ ]:
# VinVL bounding bos prediction
file_name = df_test['painting'].iloc[idx]
img_file = os.path.join(img_root,file_name+'.jpg')
img_bbox = Image.open(img_file)
img_bbox1 = ImageDraw.Draw(img_bbox)

bboxes = df_all_1.iloc[idx]['box_cords']

for x in range(len(bboxes)):
    img_bbox1.rectangle(bboxes[x], outline =colors[x+1], width=6)
    
print(idx, file_name)
print(df_all_1['emotion'].iloc[idx])
print(df_all_1['tokens'].iloc[idx])

display(img_bbox.resize((img_bbox.size[0]//3,img_bbox.size[1]//3),Image.ANTIALIAS))

In [ ]:
# VinVL + CLIP heatmap
img = Image.open(img_file)

if 'eugã¨ne' in file_name:
    file_name = file_name.replace('eugã¨ne', 'eugene')

# get heatmap
heatmap_file = df_test['painting'].iloc[idx] + '_' + df_all_1['emotion'].iloc[idx] + '.npy'
heatmap = np.load(os.path.join(heatmap_root, heatmap_file), allow_pickle=True)

w, h = img.size
heat_map_ = cv2.resize(heatmap, (w, h), interpolation = cv2.INTER_AREA)

# process heatmap
color_map = mpl_color_map.get_cmap('jet')
no_trans_heatmap = color_map(heat_map_)
heatmap_ = copy.copy(no_trans_heatmap)
heatmap_[:, :, 3] = 0.5
heatmap_ = Image.fromarray((heatmap_*255).astype(np.uint8))
no_trans_heatmap = Image.fromarray((no_trans_heatmap*255).astype(np.uint8))

heatmap_on_image = Image.new("RGBA", img.size)
heatmap_on_image = Image.alpha_composite(heatmap_on_image, img.convert('RGBA'))
heatmap_on_image = Image.alpha_composite(heatmap_on_image, heatmap_)

display(heatmap_on_image.resize((img_bbox.size[0]//3,img_bbox.size[1]//3),Image.ANTIALIAS))


In [ ]:
# WESD prediction
mask_t = df_test_pred[df_test_pred.id == df_test['painting'].iloc[idx] + '_' + df_test['emotion'].iloc[idx]].iloc[0]['prediction_matrix']
mask_ = np.asarray(mask_t)[:,6].reshape((7,7))
mask = (mask_ - np.min(mask_)) / (np.max(mask_) - np.min(mask_))
heat_map_ = cv2.resize(np.asarray(mask), (w, h), interpolation = cv2.INTER_LINEAR)

# process heatmap
color_map = mpl_color_map.get_cmap('jet')
no_trans_heatmap = color_map(heat_map_)
heatmap_ = copy.copy(no_trans_heatmap)
heatmap_[:, :, 3] = 0.5
heatmap_ = Image.fromarray((heatmap_*255).astype(np.uint8))
no_trans_heatmap = Image.fromarray((no_trans_heatmap*255).astype(np.uint8))

heatmap_on_image = Image.new("RGBA", img.size)
heatmap_on_image = Image.alpha_composite(heatmap_on_image, img.convert('RGBA'))
heatmap_on_image = Image.alpha_composite(heatmap_on_image, heatmap_)

# display(heatmap_on_image)
display(heatmap_on_image.resize((img_bbox.size[0]//3,img_bbox.size[1]//3),Image.ANTIALIAS))

In [ ]:
# APOLO annotation
paint = df_apolo.iloc[idx]['painting']
emotion = df_apolo.iloc[idx]['emotion']

mask = np.load(os.path.join('./data/apolo_pixel_map', paint + '_' + emotion + '.npy'),
                   allow_pickle=True)

img = Image.open(os.path.join('./data/artworks', paint+'.jpg')

print(idx, paint, emotion)
print(df_apolo.iloc[idx]['utterances'])

w, h = img.size
heat_map_ = cv2.resize(mask * 255, (w, h), interpolation = cv2.INTER_AREA)

# process heatmap
color_map = mpl_color_map.get_cmap('YlGn')
no_trans_heatmap = color_map(heat_map_)
heatmap_ = copy.copy(no_trans_heatmap)
heatmap_[:, :, 3] = 0.5
heatmap_ = Image.fromarray((heatmap_*255).astype(np.uint8))
no_trans_heatmap = Image.fromarray((no_trans_heatmap*255).astype(np.uint8))

heatmap_on_image = Image.new("RGBA", img.size)
heatmap_on_image = Image.alpha_composite(heatmap_on_image, img.convert('RGBA'))
heatmap_on_image = Image.alpha_composite(heatmap_on_image, heatmap_)

display(heatmap_on_image)